In [ ]:
import tensorflow as tf

In [ ]:
BUCKET = 'gs://kaggle-tsa-ml'
DAT_BUCKET = BUCKET + '/data/'
DAT_EXAMPLE = DAT_BUCKET + 'fcf366a672d2809b0d158180ecb4dd5c.aps'

IMAGE_SHAPE = [512,660,16]

In [ ]:
def read_APS(filename_queue):
    class APSrecord(object):
        pass
    result = APSrecord()

    result.height = IMAGE_SHAPE[0]
    result.width = IMAGE_SHAPE[1]
    result.depth = IMAGE_SHAPE[2]
    image_bytes = result.height * result.width * result.depth
    label_bytes = 512 
    record_bytes = image_bytes + label_bytes
    
    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    result.key, value = reader.read(filename_queue)

    # Convert from a string to a vector of uint16 that is record_bytes long.
    record_bytes = tf.decode_raw(value, tf.int16)

    # The first bytes represent the label, which we convert from uint16->int32.
    result.label = tf.cast(
        tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)

    # The remaining bytes after the label represent the image, which we reshape
    # from [depth * height * width] to [depth, height, width].
    depth_major = tf.reshape(
        tf.strided_slice(record_bytes, [label_bytes],
                         [label_bytes + image_bytes]),
        [result.depth, result.height, result.width])
    # Convert from [depth, height, width] to [height, width, depth].
    result.uint8image = tf.transpose(depth_major, [1, 2, 0])

    return result

In [ ]:
def _generate_image_and_label_batch(image, label,
                                    batch_size, shuffle,min_queue_examples=10):
    """Construct a queued batch of images and labels.
    Args:
      image: 3-D Tensor of [height, width, 3] of type.float32.
      label: 1-D Tensor of type.int32
      min_queue_examples: int32, minimum number of samples to retain
        in the queue that provides of batches of examples.
      batch_size: Number of images per batch.
      shuffle: boolean indicating whether to use a shuffling queue.
    Returns:
      images: Images. 4D tensor of [batch_size, height, width, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
  # Create a queue that shuffles the examples, and then
  # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16
    if shuffle:
        images, label_batch = tf.train.shuffle_batch(
          [image, label],
          batch_size=batch_size,
          num_threads=num_preprocess_threads,
          capacity=min_queue_examples + 3 * batch_size,
          min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch(
          [image, label],
          batch_size=batch_size,
          num_threads=num_preprocess_threads,
          capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images)

    return images, tf.reshape(label_batch, [batch_size])

In [ ]:
def gen_inputs(file_list,batch_size):
    '''
    Reads bianary files from GCS bucket 
    Input: file_list - File containing list of filenames to be read from GCS bucket.
           batch_size - Size of training data batches.
    
    Output: Batch of tensors for training.
            Batch of labels for training.
    '''
    # Read list of filenames
    with open(file_list) as fns:
        # Remove newline character form filenames 
        files = [item[:-1] for item in fns]
        #test with 5 file
        files = files[:8]
    filename_queue = tf.train.string_input_producer(files)
    
    read_input = read_APS(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)
    read_input.label.set_shape([1])
    
    ###   Simple Image pre-processing   ###
    '''
    height = IMAGE_SHAPE[0]
    width = IMAGE_SHAPE[1]
    
    
    # Here we can crop the image if desired.
    #resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image,height, width)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(resized_image)

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])
    return _generate_image_and_label_batch(float_image, read_input.label,
                                         min_queue_examples, batch_size,
                                         shuffle=False)
    '''
    return _generate_image_and_label_batch(
        reshaped_image, read_input.label, batch_size, shuffle=False)
    
    